# secrets

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt
import pandas as pd
import numpy as np
import pprint
import boto3
import json
import time
import math
import sys
import os

# os.chdir(path = '/home/steven/Documents/reddit-streaming/')
pp = pprint.PrettyPrinter(indent = 1)
base = os.getcwd()
# os.chdir(os.path.join(base, "reddit"))
# base = os.getcwd()
print("imported modules, base: {}.".format(base))

: 

## read creds json

In [2]:
os.listdir(base)

['create_athena_table.sql',
 'send_messages.ipynb',
 'spark-streaming-test.ipynb',
 'producer-multiple-topics.ipynb',
 'scripts',
 'reddit-producer-multiple-topics.ipynb',
 'reddit',
 'reddit-producer-test.ipynb',
 'main.ipynb',
 'main.py',
 '__init__.py',
 'read_table.ipynb',
 'secrets.ipynb',
 'setup.py',
 'config.yaml',
 '__pycache__']

In [3]:
with open("creds.json", "r") as f:
    creds = json.load(f)
    f.close()

# pp.pprint(creds)

FileNotFoundError: [Errno 2] No such file or directory: 'creds.json'

## spark

In [16]:
spark_host = "spark-master" 
kafka_host = "kafka" 
subreddit = "cosmos"
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]

spark = SparkSession.builder.appName("cosmos") \
    .master("spark://{}:7077".format(spark_host)) \
    .config("spark.scheduler.mode", "FAIR") \
    .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
    .config("spark.executor.memory", "4096m") \
    .config("spark.executor.cores", "4") \
    .config("spark.streaming.concurrentJobs", "4") \
    .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
    .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.SparkContext
print("created spark successfully.")

22/08/22 16:55:10 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.io.FileNotFoundException: File file:/opt/workspace/events/cosmos does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.deploy.history.EventLogFileWriter.requireLogBaseDirAsDirectory(EventLogFileWriters.scala:77)
	at org.apache.spark.deploy.history.SingleEventLogFileWriter.start(EventLogFileWriters.scala:221)
	at org.apache.spark.scheduler.EventLoggingListener.start(EventLoggingListener.scala:83)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:610)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at